# Decision Trees 

Classification, regression, multioutput tasks

* require very little data preparation. In particular, they don't require feature scaling or centering at all
* *CART* algorithm produces only *binary trees*, *ID3* produces trees with more branches 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Training-and-visualizing-a-decision-tree" data-toc-modified-id="Training-and-visualizing-a-decision-tree-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Training and visualizing a decision tree</a></span><ul class="toc-item"><li><span><a href="#An-example-of-a-decision-tree---tree:DecisionTreeClassifier" data-toc-modified-id="An-example-of-a-decision-tree---tree:DecisionTreeClassifier-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>An example of a decision tree - <code>tree:DecisionTreeClassifier</code></a></span></li><li><span><a href="#Visualization-of-a-tree---tree:export_graphviz" data-toc-modified-id="Visualization-of-a-tree---tree:export_graphviz-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Visualization of a tree - <code>tree:export_graphviz</code></a></span></li><li><span><a href="#Making-predictions---.predict_proba(),.predict()" data-toc-modified-id="Making-predictions---.predict_proba(),.predict()-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Making predictions - <code>.predict_proba()</code>,<code>.predict()</code></a></span></li></ul></li><li><span><a href="#The-CART-training-algorithm" data-toc-modified-id="The-CART-training-algorithm-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>The CART training algorithm</a></span><ul class="toc-item"><li><span><a href="#CART-cost-function-for-classification" data-toc-modified-id="CART-cost-function-for-classification-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>CART cost function for classification</a></span></li><li><span><a href="#Computational-complexity" data-toc-modified-id="Computational-complexity-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Computational complexity</a></span></li><li><span><a href="#Gini-impurity-or-entropy" data-toc-modified-id="Gini-impurity-or-entropy-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Gini impurity or entropy</a></span></li></ul></li><li><span><a href="#Regularization-hyperparameters" data-toc-modified-id="Regularization-hyperparameters-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Regularization hyperparameters</a></span><ul class="toc-item"><li><span><a href="#pruning-algorithm" data-toc-modified-id="pruning-algorithm-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>pruning algorithm</a></span></li></ul></li><li><span><a href="#Regression---tree:DecisionTreeRegressor" data-toc-modified-id="Regression---tree:DecisionTreeRegressor-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Regression - <code>tree:DecisionTreeRegressor</code></a></span><ul class="toc-item"><li><span><a href="#CART-cost-function-for-regression" data-toc-modified-id="CART-cost-function-for-regression-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>CART cost function for regression</a></span></li></ul></li><li><span><a href="#Instability" data-toc-modified-id="Instability-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Instability</a></span></li></ul></div>

## Training and visualizing a decision tree 

### An example of a decision tree - `tree:DecisionTreeClassifier`

In [9]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:,2:]
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

### Visualization of a tree - `tree:export_graphviz`

In [12]:
from sklearn.tree import export_graphviz

export_graphviz(
            tree_clf,
            out_file="./iris_tree.dot",
            feature_names=iris.feature_names[2:],
            class_names=iris.target_names,
            rounded=True,
            filled=True
        )

then convert this *.dot* file to PDF or PNG using the `dot` command-line tool from the `graphviz` package
```[bash]
$ dot -Tpng iris_tree.dot-o iris_tree.png
```

### Making predictions - `.predict_proba()`,`.predict()`

the explanation of attributes
* `samples`: counts how many training instances it applies to
* `value`: how many training instances of each class this node applied to
* `gini`: the impurity
$$G_i=1-\sum_{k=1}^{n}p_{i,k}^2$$
    * (`gini=0` ) the node is "pure"
    * $p_{i,k}$ is the ratio of class $k$ instances among the training instances in the $i^{th}$ node

In [13]:
tree_clf.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [15]:
tree_clf.predict([[5,1.5]])

array([1])

## The CART training algorithm

the *Classification And Regression Tree* (CART), a *greedy* algorithm

<span class="burk">**Finding the optimal tree is known to be an *NP-Complete* problem, which requires $O(\exp(m))$ time.**</span>

### CART cost function for classification 

$$J(k,t_k)=\frac{m_{left}}{m}G_{left}+\frac{m_{right}}{m}G_{right}$$

* $G_*$ measures the impurity of the * subset
* $m_*$ is the number of instances in the * subset

It stops recursing once
* it reaches the maximum depth `max_depth`
* it cannot find a split that will reduce impurity

### Computational complexity 

prediction ~ $O(\log_2(m))$

training ~ $O(n\times m\log(m))$ or less if `max_features` is set

### Gini impurity or entropy

Entropy
$$H_i=-\sum_{k=1,p_{i,k}\neq 0}^{n}p_{i,k}\log(p_{i,k})$$

* Gini impurity is slightly faster to compute, so it is a good default
* Gini impurity tends to isolate the most frequent class in its won branch of the tree, while entropy tends to produce slightly more balanced trees.

## Regularization hyperparameters

* `max_depth`: default is `None` which leads to *nonparametric model*. Set it to restrict the maximum depth of the Decision tree.
* `min_sample_split`: the minimum number of samples a node must have before it can be split
* `min_samples_leaf`: the minimum number of samples a leaf node must have.
* `min_weight_fraction_leaf`: expressed as a fraction of the total number of weighted instances.
* `max_leaf_nodes`: maximum number of leaf nodes
* `max_features`: maximum number of features that are evaluated for splitting at each node

increasing `min_*` or reducing `max_*` results in regularization

### pruning algorithm
first training the decision tree without restrictions, then deleting unnecessary nodes.

A node whose children are all leaf nodes is considered unnecessary if the purity improvement it provides is not *statistically significant*

## Regression - `tree:DecisionTreeRegressor`

In [16]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

Minimize the MSE

### CART cost function for regression 

$$J(k,t_k)=\frac{m_{left}}{m}MSE_{left}+\frac{m_{right}}{m}MSE_{right}$$

where 
* $MSE_* = \sum_{i\in node}(\hat{y}_{node}-y^{(i)})^2$
* $\hat{y}_{node}=\frac{1}{m_{node}}\sum_{i\in node}y^{(i)}$

## Instability

* Decision trees love orthogonal decision boundaries, which makes them sensitive to training set rotation.
    * use *PCA* (chapter 8) to get a better orientation of the training data
* very sensitive to small variations in the training data
* stochastic training algorithm, 
    * set `random_state` to get the same model
    * use *Random Forests* model